In [9]:
import cv2
import numpy as np
import serial
import time
import torch
from PIL import ImageFont, ImageDraw, Image

# 화면에 한글을 표기하기 위한 함수
def myPutText(src, text, pos, font_size, font_color):
    img_pil = Image.fromarray(src)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype('fonts/gulim.ttc', font_size)
    draw.text(pos, text, font = font, fill = font_color)
    return np.array(img_pil)

# 화면이 터치되면 감지하고 아래 동작을 실행
def mouse_handler(event, x, y, flags, param):
    global mouse_loc
    global state
    if event == cv2.EVENT_LBUTTONDOWN: # 마우스 왼쪽 버튼 누름 or 화면 터치
        if state == 0:
            ser.write(bytes([0b10000000])) # 정방향 회전 명령 전달
            while(not ser.readable()): pass # Arduino로부터 완료 신호 수신받을때 까지 기다리기, 아두이노가 없다면 주석처리
            response = ser.read(1)           # 아두이노가 없다면 주석처리
            state = 1 # 다음 상태로 넘어가기
        elif state == 2 or state == 3:
            ser.write(bytes([0b00000000])) # 정방향 회전 명령 전달
            while(not ser.readable()): pass # Arduino로부터 완료 신호 수신받을때 까지 기다리기, 아두이노가 없다면 주석처리
            response = ser.read(1)           # 아두이노가 없다면 주석처리
            state = 0
                
    elif event == cv2.EVENT_LBUTTONUP: # 마우스 왼쪽 버튼 땜
        pass
        
# Color
WHITE = (255, 255, 255)
BLUE = (255, 0, 0)
GREEN = (0, 255, 0)
YELLOW = (0, 255, 255)
BLACK = (0,0,0)

state = 0

# read image
logo1 = cv2.imread('logo1.png')
logo2 = cv2.imread('logo2.png')
logo3 = cv2.imread('logo3.png')

cup = cv2.imread('cup_front.png')
QR = cv2.imread('QR.png')

h1, w1, _ = logo1.shape
h_cup, w_cup, _ = cup.shape

# Initial Image
init_img = np.zeros((640, 640, 3), dtype = np.uint8)
init_img[:,:] = (255, 255, 255)
init_img = myPutText(init_img, '화면의 아무 곳이나 터치하여 다음 단계로 넘어가세요', (75, 600), 20, BLACK)

img = init_img

# Window
window_name = 'Frame'
cv2.namedWindow(window_name) 

# Load the video
cap = cv2.VideoCapture(0)

# For USART Communication, 아두이노가 없다면 주석처리
ser = serial.Serial(port = 'COM5', baudrate = 9600)

# For object detection
# Load the YOLOv5 model with 'best.pt' weights
model = torch.hub.load('ultralytics/yolov5', 'custom', path='brand2_best.pt')

# Brand list
brand_list = ['starbucks']
object_list = []

# Set device
# device = 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device).eval()

while True:
    cv2.setMouseCallback(window_name, mouse_handler)
    if state == 0:
        img = myPutText(img, '아래 브랜드에 해당하는 컵을 놔주세요', (50, 20), 30, BLACK)
        img[80:80+h1, 14:14+w1] = logo1
        img[80:80+h1, 14+w1: 14+w1*2] = logo2
        img[80:80+h1, 14+w1*2: 14+w1*3] = logo3
        img = myPutText(img, '아래 사진 처럼 로고가 보이게 컵을 놔주세요', (20, 220), 30, BLACK)
        img[300:420,260:380] = cup
        
    elif state == 1:
        #img = myPutText(img, '컵을 확인할 때까지 기다려주세요', (90, 20), 30, BLACK)
        time.sleep(0.5)
        start_pos = time.time()
        ret, frame = cap.read()
        if not ret:
            print('영상을 사용할 수 없음')
            break
        
        
        # Perform inference on an image
        results = model(frame)
        # Access the detected objects and their labels
        objects = results.pandas().xyxy[0]
        # Print the detected objects and their labels
        for _, obj in objects.iterrows():
            class_name = obj['name']
            confidence = obj['confidence']
            object_list.append(class_name)
            IsBrandLeast = True
            if confidence > 0.6 and (class_name in brand_list):
                # print('Class:', class_name)
                # print('Confidence:', confidence)
                cv2.rectangle(frame, (int(obj['xmin']), int(obj['ymin'])), (int(obj['xmax']), int(obj['ymax'])), (0, 255, 0), 2)
                # Display the class name and confidence on the frame
                label = f'{class_name}: {confidence:.2f}'
                cv2.putText(frame, label, (int(obj['xmin']), int(obj['ymin']) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        
        if 'starbucks' in object_list:
            state = 2
            ser.write(bytes([0b11000000])) # 정방향 회전 명령 전달
            while(not ser.readable()): pass # Arduino로부터 완료 신호 수신받을때 까지 기다리기, 아두이노가 없다면 주석처리
            response = ser.read(1)           # 아두이노가 없다면 주석처리
        else:
            state = 3
            # ser.write(bytes([0b00000000])) # 역방향 회전 명령 전달
            # while(not ser.readable()): pass # Arduino로부터 완료 신호 수신받을때 까지 기다리기, 아두이노가 없다면 주석처리
            # response = ser.read(1)           # 아두이노가 없다면 주석처리
        
        object_list = []
        latency = time.time() - start_pos
        print("Latency : {}ms".format(int(1000*latency)))
        
    elif state == 2:
        img = myPutText(img, '아래 QR 코드로 환급 받아가세요', (90, 20), 30, BLACK)
        img[120:600, 80:560] = frame[:, 80:560]
        img[120:240, 80:200] = cv2.resize(QR, (120,120))
    elif state == 3:
        img = myPutText(img, '잘못된 브랜드거나 위치가 올바르지 않습니다', (20, 20), 30, BLACK)
        img = myPutText(img, '컵을 다시 확인해 주세요', (150, 70), 30, BLACK)
        img[120:600, 80:560] = frame[:, 80:560]
        
            
    cv2.imshow(window_name, img)
    img = init_img
    if cv2.waitKey(1) == ord('q'):
        break
        
ser.close()    
cap.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\danie/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-29 Python-3.7.12 torch-1.10.0+cu102 CUDA:0 (GeForce MX250, 2048MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


Latency : 418ms
Latency : 45ms
Latency : 49ms
Latency : 53ms
Latency : 45ms
Latency : 47ms
Latency : 47ms
Latency : 47ms
Latency : 53ms
Latency : 50ms
Latency : 48ms


In [6]:
time.sleep(0.5)